In [1]:
from keras.models import Model
from keras.layers import Conv2D,Conv2DTranspose,Reshape,Flatten,Input,Dense
from keras.datasets import mnist
from keras import backend as k
import numpy as np
from PIL import Image

Using TensorFlow backend.


In [3]:
(x_train,_),(x_test,_)=mnist.load_data()

image_size=x_train.shape[1]

input_size=(image_size,image_size,1)

x_train=np.reshape(x_train,[-1,image_size,image_size,1])
x_train=x_train.astype('float32')/255

x_test=np.reshape(x_test,[-1,image_size,image_size,1])
x_test=x_test.astype('float32')/255

batch_size=32
kernels=3
latent_dim=64
layer_filters=[32,64]

noise=np.random.normal(loc=.5,scale=.5,size=x_train.shape)
x_train_noisy=x_train+noise

noise=np.random.normal(loc=.5,scale=.5,size=x_test.shape)
x_test_noisy=x_test+noise

inputs=Input(shape=input_size,name='encoder_nput')
x=inputs

x_train_noisy=np.clip(x_train_noisy,0.,1.)
x_test_noisy=np.clip(x_test_noisy,0.,1.)

for filter in layer_filters:
    x=Conv2D(kernel_size=kernels,padding='same',activation='relu',filters=filter)(x)
    
shape=k.int_shape(x_train)
x=Flatten()(x)

latent=Dense(latent_dim,name='latent_nput')(x)

encoder=Model(inputs,latent,name='encoder_tput')
encoder.summary()

latent_inputs=Input(shape=(latent_dim,),name='decoderi_put')

x=Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
x=Reshape((shape[1],shape[2],shape[3]))(x)


for filter in layer_filters[::-1]:
    x=Conv2DTranspose(filters=filter,kernel_size=kernels,padding='same',activation='relu',strides=2)(x)
    
output=Conv2DTranspose(filters=1,kernel_size=kernels,padding='same',activation='softmax',name='decodr_output')(x)



decoder=Model(latent_inputs,output,name='decoder')

decoder.summary()



autoencoder=Model(inputs,decoder(encoder(inputs)),name='autoencoder')

autoencoder.summary()

autoencoder.compile(loss='categorical_crossentropy',optimizer='adam')

autoencoder.fit(x_train,x_train,epochs=10,batch_size=batch_size,validation_data=(x_test_noisy,x_test))


x_decoded=autoencoder.predict(x_test_noisy)


rows,cols=3,9
nums=rows*cols
imgs=np.concatenate([x_train_noisy[:nums],x_test_noisy[:nums],x_decoded[:nums]])
imgs=np.reshape((rows*3,cols,image_size,image_size))
imgs=np.vstack(imgs,rows,axis=1)
imgs=np.reshape((rows*3,-1,image_size,image_size))
imgs=np.vstack([np.hstack[i] for i in imgs])
imgs=(imgs*255).astype('np.unit8')
plt.figure()
plt.axis('off')
plt.title('original images   1st row',
         'corrupted images 2nd row',
         'decoded images 3rd row')
plt.imshow(imgs,cmap='grey',interpolation='none')

Image.fromarray(imgs).save('corrupted_decoded.png')
plt.show()

Model: "encoder_tput"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_nput (InputLayer)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
flatten_2 (Flatten)          (None, 50176)             0         
_________________________________________________________________
latent_nput (Dense)          (None, 64)                3211328   
Total params: 3,230,144
Trainable params: 3,230,144
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)               

ValueError: Error when checking target: expected decoder to have shape (112, 112, 1) but got array with shape (28, 28, 1)

In [15]:
x_test_noisy.shape

(10000, 28, 28, 1)

In [17]:
x_train.shape

(60000, 28, 28, 1)